In [1]:

'''
    Für farbige Ausgaben.
'''
def coloring(string,color="normal",style="normal"):
    col = "0"
    sty = "2"
    if color == "blink":
        col = "5"
    elif   color == "red":
        col = "31"
    elif color == "green":
        col = "32"
    elif color == "yellow":
        col = "33"
    elif color == "blue":
        col = "34"
    elif color == "purple":
        col = "35"
    elif color == "blueL":
        col = "94"


    if style == "bold":
        sty = "1"
    elif style == "italic":
        sty = "3"
    elif style == "curl":
        sty = "4"
    elif style == "blink1":
        sty = "5"
    elif style == "blink2":
        sty = "6"
    elif style == "selected":
        sty = "7"

    return "\x1b[{}m\x1b[{}m{}\x1b[0m".format(col,sty,string)

In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf



'''
 *** Infos ***

Input Data Columns:

    'confirmed', 'deaths', 'recovered', 'vaccines', 'people_vaccinated','people_fully_vaccinated', 'school_closing', 'workplace_closing',
    'cancel_events', 'gatherings_restrictions', 'transport_closing', 'stay_home_restrictions', 'internal_movement_restrictions',
    'internal_movement_restrictions', 'information_campaigns', 'testing_policy', 'contact_tracing', 'facial_coverings','vaccination_policy',
    'elderly_people_protection', 'population', 'cfr', 'cases_per_population', 'incidence'


Eingabe: numpy.array
    Shape:  (SEQUENCE_COUNT, INPUT_STEPS, INPUT_COLS)  

Ausgabe: numpy.array
    Shape:  (SEQUENCE_COUNT, OUTPUT_STEPS, 1)
   
'''


def get_relevant_cols(df):
    return df[['confirmed', 'deaths' ,"recovered"
        ,"vaccines","people_vaccinated","people_fully_vaccinated"
        ,"school_closing","workplace_closing","cancel_events"
        ,"gatherings_restrictions","transport_closing"
        ,"stay_home_restrictions","internal_movement_restrictions"
        ,"internal_movement_restrictions","information_campaigns","testing_policy"
        ,"contact_tracing","facial_coverings"
        ,"vaccination_policy","elderly_people_protection","population","cfr","cases_per_population","incidence"]]


input_scaler = MinMaxScaler(feature_range=(0, 1))
output_scaler = MinMaxScaler(feature_range=(0, 1))

predict_model = tf.keras.models.load_model("model.h5", compile=False)
predict_model.compile(optimizer='adam', loss='mse')

## Einlesen der Ein- und Ausgabeinfos des Models
INPUT_STEPS = predict_model.get_build_config()["input_shape"][1]
INPUT_COLS = predict_model.get_build_config()["input_shape"][2]
OUTPUT_STEPS = predict_model.get_config()["layers"][2]["config"]["n"]

SEQUENCE_COUNT = 1 # Wie viele Sequenzen (jeweils INPUT_STEPS lang und mit INPUT_COLS Spalten) werden eingegeben

selected_lk = "LK Ahrweiler"
lk_data_index = INPUT_STEPS*SEQUENCE_COUNT

## Einlesen der Daten
data_csv = pd.read_csv('../data/clean/df_weekly_incidence.csv')
lk_data_full= data_csv[data_csv.administrative_area_level_3 == selected_lk]

predict_year_weeks = lk_data_full[["year","week"]].iloc[lk_data_index:lk_data_index+OUTPUT_STEPS]


## Extrahieren der für das Netwerk wichtigen Spalten
lk_data = get_relevant_cols(lk_data_full)

## Normieren der Daten
lk_input_data = lk_data.to_numpy()[:,:-1]
lk_input_data = input_scaler.fit_transform(lk_input_data)

lk_output_data = lk_data.to_numpy()[:,-1].reshape(lk_data.shape[0],1)
output_scaler.fit(lk_output_data)


## Eingabesequenz extrahieren und reshapen 
data_to_predict = lk_input_data[0:lk_data_index,:]
data_to_predict = data_to_predict.reshape(SEQUENCE_COUNT,INPUT_STEPS,INPUT_COLS)

## Prediction 
prediction_result = predict_model.predict(data_to_predict,verbose=0).reshape(SEQUENCE_COUNT*OUTPUT_STEPS,1)
prediction_result = output_scaler.inverse_transform(prediction_result)[:,0]


result_df = pd.DataFrame(data={
    "Jahr":predict_year_weeks.year.to_numpy(),
    "Woche":predict_year_weeks.week.to_numpy(),
    "Inzidenz":prediction_result})

print("Vorhersage für {} ab Woche {} in {}".format(coloring(selected_lk,style="bold"),predict_year_weeks.week.iloc[0],predict_year_weeks.year.iloc[0]))
# display("Vorhersage für '{}' ab Woche {} in {}".format(selected_lk,predict_year_weeks.week.iloc[0],predict_year_weeks.year.iloc[0]))
# display(result_df)

column_string = "  {}   | {}  |  {} ".format(coloring("Jahr","blue","bold"),coloring("Woche","blueL","bold"),coloring("Inzidenz","purple","bold"))
print("{:-<32s}\n{}\n{:-<32s}".format("",column_string,""))
for i, row in result_df.iterrows():
    print(" {: ^20} | {: ^19} | {: ^22} ".format(coloring(f"{row.Jahr:.0f}","blue"),coloring(f"{row.Woche:.0f}","blueL"),coloring(f"{row.Inzidenz:.1f}","purple"))) 


AttributeError: 'Sequential' object has no attribute 'get_build_config'

In [11]:

predict_model = tf.keras.models.load_model("model.h5", compile=False)
opt = tf.keras.optimizers.Adam(weight_decay=True)
predict_model.compile(optimizer=opt, loss='mse')

predict_model.get_build_config()


TypeError: Unexpected keyword argument passed to optimizer: weight_decay. Allowed kwargs are {'lr', 'clipnorm', 'clipvalue', 'decay', 'global_clipnorm'}.